In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import json

data = pd.read_csv('train.csv')
data.head()


In [ ]:
print data.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data,
    data.left,
    test_size=0.2,
    random_state=0)

print len(X_train)

In [ ]:
#map_range_sl = lambda x : '<=0.37' if x <= 0.37 else ('>0.37 and <=0.51' if x <= 0.51 else ('>0.51 and <=0.64' if x<=0.64 else ('>0.64 and <=0.76' if x<=0.76 else ('>0.76 and <=0.87' if x<=0.87 else '>0.87'))))
map_range_sl = lambda x: '<=0.17' if x<=0.17 else ('0.18 to 0.29' if x <= 0.29 else ('0.30 to 0.39' if x<=0.39 else ('0.40 to 0.46' if x<=0.46 else ('0.47 to 0.53' if x<=0.53 else ('0.54 to 0.6' if x<=0.6 else ('0.61 to 0.67' if x<=0.67 else ('0.68 to 0.74' if x<=0.74 else ('0.75 to 0.8' if x<=0.8 else ('0.81 to 0.87' if x<=0.87 else ('0.88 to 0.94' if x<=0.94 else '>0.94'))))))))))
X_train.satisfaction_level = X_train.satisfaction_level.apply(map_range_sl)
X_test.satisfaction_level = X_test.satisfaction_level.apply(map_range_sl)

# map_range_le = lambda x : '<=0.52' if x <= 0.52 else ('>0.52 and <=0.61' if x <= 0.61 else ('>0.61 and <=0.72' if x<=0.72 else ('>0.72 and <=0.83' if x<=0.83 else ('>0.83 and <=0.91' if x<=0.91 else '>0.91'))))
map_range_le = lambda x: '<=0.44' if x<=0.44 else ('0.45 to 0.51' if x <= 0.51 else ('0.52 to 0.58' if x<=0.58 else ('0.59 to 0.65' if x<=0.65 else ('0.66 to 0.71' if x<=0.71 else ('0.72 to 0.78' if x<=0.78 else ('0.79 to 0.86' if x<=0.86 else ('0.87 to 0.93' if x<=0.93 else '>0.93'))))))) 
X_train.last_evaluation = X_train.last_evaluation.apply(map_range_le)
X_test.last_evaluation = X_test.last_evaluation.apply(map_range_le)

# map_range_np = lambda x : '<=3' if x <=3 else ('>3 and <=6' if x<=6 else '>6')
map_range_np = lambda x: '<=3' if x<=3 else '>3'
X_train.number_project = X_train.number_project.apply(map_range_np)
X_test.number_project = X_test.number_project.apply(map_range_np)

# map_range_amh = lambda x : '<=156' if x <=156 else ('>156 and <=199' if x<=199 else ('>199 and <=245' if x<=245 else '>245'))
map_range_amh = lambda x: '<=173' if x<=173 else ('174 to 229' if x<=229 else '>229')
X_train.average_montly_hours = X_train.average_montly_hours.apply(map_range_amh)
X_test.average_montly_hours = X_test.average_montly_hours.apply(map_range_amh)

map_range_ts = lambda x : '<=3' if x <=3 else ('>3 and <=6' if x<=6 else '>6')
X_train.time_spend_company = X_train.time_spend_company.apply(map_range_ts)
X_test.time_spend_company = X_test.time_spend_company.apply(map_range_ts)

In [ ]:
X_train.head()

In [ ]:
def create_tree_node(value, is_leaf,number,height):
    new_node = dict()
    new_node['value'] = value
    new_node['is_leaf'] = is_leaf
    new_node['children'] = dict()
    new_node['height'] = height
    new_node['number'] = number
    return new_node

In [ ]:
def display_tree(t,index):
    print t
    tabs = ''
    for i in range(0,index):
        tabs+='\t'
    print tabs+str(t['value'])
    for c in t['children']:
        i = (index+1)
        display_tree(c,i)

In [ ]:
def compute_impurity(pos_ratio,neg_ratio,impurity_function):
    if impurity_function == 'entropy':
        if pos_ratio == 0 or pos_ratio == 1:
            return 0
        else:
            return -((pos_ratio*np.log(pos_ratio)) + (neg_ratio*np.log(neg_ratio)))
    elif impurity_function == 'gini_index':
        return (2*pos_ratio*neg_ratio)
    elif impurity_function == 'misclassification_rate':
        return min(pos_ratio,neg_ratio)
    return 0

In [ ]:
def print_tree(curr_data,curr_parent_object,curr_condition, exclude_cols,decision_tree,number_of_nodes,height):
    
    curr_parent_str = curr_parent_object['value']
    if curr_parent_str not in exclude_cols:
        exclude_cols.append(curr_parent_str)
    #print 'Current Parent object : '+str(curr_parent_object)
    #print 'Current Condition : '+str(curr_condition)
    m = len(curr_data)
    G = dict()
    for col in curr_data:
        if col in exclude_cols:
            continue
        I = 0.0
        col_pos = 0
        col_neg = 0
        int_info = 0
        for categ in curr_data[col].unique():
            #print 'col :: '+col+' ; categ :: '+str(categ)
            #print curr_data.groupby([col])['left'].value_counts()[categ]
            pos = 0
            neg = 0
            if 1 in curr_data.groupby([col])['left'].value_counts()[categ]: 
                pos = curr_data.groupby([col])['left'].value_counts()[categ][1]
            if 0 in curr_data.groupby([col])['left'].value_counts()[categ]:
                neg = curr_data.groupby([col])['left'].value_counts()[categ][0]
            col_pos += pos
            col_neg += neg
            pos_ratio = float(pos)/float(pos+neg)
            neg_ratio = float(neg)/float(pos+neg)
            impurity = compute_impurity(pos_ratio,neg_ratio, 'entropy')
            Si_S = float(pos+neg)/float(m) 
            I += Si_S*impurity
            int_info += Si_S*np.log(Si_S)
        col_pos_ratio = float(col_pos)/float(col_pos+col_neg)
        col_neg_ratio = float(col_neg)/float(col_pos+col_neg)
        E = compute_impurity(col_pos_ratio,col_neg_ratio,'entropy')
        G[col] = ((E-I)/(-1*int_info))
    
    if not G:
        if curr_parent_str == 'dummy_parent':
            return decision_tree
        for categ in curr_data[col].unique():
            if 0 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition] and 1 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                if curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition][0] > curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition][1]:
                    #print 'Left'
                    number_of_nodes += 1
                    new_node = create_tree_node('0',True, number_of_nodes, height)
                    curr_parent_object['children'][curr_condition] = new_node
                else:
                    number_of_nodes += 1
                    new_node = create_tree_node('1',True, number_of_nodes, height)
                    curr_parent_object['children'][curr_condition] = new_node
            elif 0 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                number_of_nodes += 1
                new_node = create_tree_node('0',True, number_of_nodes, height)
                curr_parent_object['children'][curr_condition] = new_node
            elif 1 in curr_data.groupby([curr_parent_str])['left'].value_counts()[curr_condition]:
                number_of_nodes += 1
                new_node = create_tree_node('1',True, number_of_nodes, height)
                curr_parent_object['children'][curr_condition] = new_node
            else:
                number_of_nodes += 1
                new_node = create_tree_node('0',True, number_of_nodes, height)
                curr_parent_object['children'][curr_condition] = new_node
        return (decision_tree,number_of_nodes)
    
    max_col = max(G, key=G.get)
    new_node = None
    
    if new_node is None:
        number_of_nodes += 1
        new_node = create_tree_node(max_col,False, number_of_nodes, height)
    if decision_tree is None:
        decision_tree = new_node
    else:
        curr_parent_object['children'][curr_condition] = new_node
    
    
    if new_node['is_leaf']:
        return (decision_tree,number_of_nodes)
    
    
    temp_exclude_cols = exclude_cols[:]
    for cond in curr_data[max_col].unique():
        if 0 in curr_data.groupby([max_col])['left'].value_counts()[cond]:
            neg_count = curr_data.groupby([max_col])['left'].value_counts()[cond][0]
        else:
            neg_count = 0
        if 1 in curr_data.groupby([max_col])['left'].value_counts()[cond]:
            pos_count = curr_data.groupby([max_col])['left'].value_counts()[cond][1]
        else:
            pos_count = 0
        new_node['pos_count'] = pos_count
        new_node['neg_count'] = neg_count
        decision_tree,number_of_nodes = print_tree(curr_data.loc[curr_data[max_col]==cond].copy() , new_node, cond,temp_exclude_cols,decision_tree,number_of_nodes,(height+1))
    
    return (decision_tree,number_of_nodes)

In [ ]:
decision_tree = None
dummy_node = dict()
dummy_node['value'] = 'dummy_parent'
decision_tree,number_of_nodes = print_tree(X_train.copy(),dummy_node,'dummy_condition',['left'],decision_tree,0,1)
print number_of_nodes

In [ ]:
def validate_row(row,tree_node,max_height):
    if tree_node['is_leaf']:
        if row['left'] == 1 and tree_node['value'] == '1':
            #true positive
            return 2
        elif row['left'] == 1 and tree_node['value'] == '0':
            #false negative
            return -1
        elif row['left'] == 0 and tree_node['value'] == '1':
            #false positive
            return 0
        elif row['left'] == 0 and tree_node['value'] == '0':
            #true negative
            return 1
    if tree_node['height'] >= max_height:
        pc = tree_node['pos_count']
        nc = tree_node['neg_count']
        if pc <= nc and row['left']==1:
            #false negative
            return -1
        elif pc<=nc and row['left']==0:
            #true negative
            return 1
        elif row['left'] == 1:
            #true positive
            return 2
        else:
            #false positive
            return 0
            
    key = tree_node['value']
    val = row[key]
    if val not in tree_node['children']:
        #error considered as true negative
        return 1
    else:
        return validate_row(row,tree_node['children'][val],max_height)

In [ ]:
def get_accuracy(X,tree,max_height):
    m = len(X)
    hits = 0
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for index,row in X.iterrows():
        verdict = validate_row(row,tree,max_height)
        if verdict == -1:
            fn += 1
        elif verdict == 0:
            fp += 1
        elif verdict == 1:
            hits += 1
            tn += 1
        elif verdict == 2:
            hits += 1
            tp += 1
    
    accuracy = float(hits)/float(m)
    return accuracy

In [ ]:
heights = list()
train_accuracies = list()
for h in range(1,11):
    acc = get_accuracy(X_train,decision_tree,h)
    heights.append(h)
    train_accuracies.append(acc)

test_accuracies = list()
for h in range(1,11):
    acc = get_accuracy(X_test,decision_tree,h)
    test_accuracies.append(acc)
# print train_accuracies
df = pd.DataFrame({'Train Performance': train_accuracies,'Test Performance': test_accuracies}, index=heights)
lines = df.plot.line()